### Imports

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from langchain.prompts import PromptTemplate
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score

from utils.loader import llm, emb_fn, strings
from utils.utils import prepare_data, set_all_seeds, save_predictions
from utils.optimizer import Protegi

In [2]:
set_all_seeds(42)

### Data

In [3]:
prepare_data('spam', '/home/umbilnm/python_ml/AutomatizationPromptEngeneering/data/spam.csv', [100, 1000]) 
df = pd.read_csv('/home/umbilnm/python_ml/AutomatizationPromptEngeneering/data/spam_100.csv').drop(
    columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4']
)
messages = df['v2']

### Model

In [4]:
# template = strings['templates']['prediction_template']
# template = PromptTemplate(template=template, input_variables=['message'])

# preds = []
# for message in tqdm(messages):
#     prediction = llm.invoke(template.format(message=message)).content
#     preds.append(prediction)
#     df['predictions'] = 'ham' if prediction=='No' else 'spam'

In [7]:
df['predictions'] = preds
df['predictions'] = df['predictions'].apply(lambda x: 0 if x=='No' else 1)
df['v1'] = df['v1'].apply(lambda x: 0 if x=='ham' else 1)

NameError: name 'preds' is not defined

In [4]:
preds_df = pd.read_csv('data/predistions/first_predictions.csv')
df = pd.read_csv('data/spam_100.csv').drop(columns=[f'Unnamed: {i}' for i in range(2, 5)])
df['preds'] = preds_df['predictions']
df['v1'] = df['v1'].apply(lambda x: 1 if x=='spam' else 0)

### Metrics

In [66]:
accuracy = accuracy_score(df['v1'], df['predictions'])
recall = recall_score(df['v1'], df['predictions'])
f1 = f1_score(df['v1'], df['predictions'])
precision = precision_score(df['v1'], df['predictions'])
print(f'Accuracy = {accuracy:.2f}\nRecall = {recall:.2f}\nPrecision = {precision:.2f}\nF1 = {f1:.2f}')

Accuracy = 0.82
Recall = 1.00
Precision = 0.40
F1 = 0.57


### Protegi

In [5]:
prot = Protegi()
prompt = strings['templates']['prediction_template']
error_str = prot._sample_errors_string(df=df, true_col='v1', pred_col='preds', text_col='v2')
grad = prot._get_gradient(error_str)
new_prompts = prot.apply_gradient(prompt=prompt, error_str=error_str, feedback_str=grad, steps_per_gradient=3)

In [6]:
print(new_prompts.content)

1. "Classify the following message as spam or not spam. 
Message: {message}
"

2. "Is the following message spam? 
Message: {message}
"

3. "Identify if the message below is spam. 
Message: {message}
"

I will test these prompts and see if they improve the accuracy of the zero-shot classifier.


In [8]:
print(grad.content)

<START>
1. The prompt may not be capturing the full context of the message, leading to misinterpretation of the content as spam.
<END>
<START>
2. The language and tone of the message may be misleading, causing the model to incorrectly classify it as spam.
<END>
<START>
3. The prompt may not be effectively training the model to differentiate between spam and non-spam messages, leading to inaccurate predictions.
<END>
